In [23]:
import json
import tensorflow as tf
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import argparse
import os
import word2vec
import visualizer as VS
import eval_utils as EU
import numpy as np
import pandas as pd
from gensim.models import Word2Vec, KeyedVectors
import sys

%reload_ext autoreload
%autoreload 2

In [153]:
embedding_storage = {
    "node2vec": '../node2vec/emb/',
    "ProNE": '../ProNE/emb/'
}

with open("../embedding_config.json", "r") as jsonfile:
    embeddding_config = json.load(jsonfile)
num_bins = embeddding_config["num_bins"]
test_size = embeddding_config["test_size"]
task = "kraken"
method = "ProNE"

In [154]:
with open("../data/data_config.txt", "r") as jsonfile:
    data_config = json.load(jsonfile)
with open("../data/strategies/" + task + ".txt", "r") as jsonfile:
    strategies = json.load(jsonfile)
config = data_config[task]
location = config["location"]
target_file = config["target_file"]
location_processed = config["location_processed"]
target_column = config["target_column"]

In [155]:
# Load data
trimmed_table = pd.read_csv(os.path.join("../", location_processed), sep=',', encoding='latin')
full_table = pd.read_csv(os.path.join("../", location + target_file), sep=',', encoding='latin')

Y = full_table[target_column]
if task in ["kraken", "financial", "genes"]:
    Y = pd.Categorical(Y).codes

# Set embeddings that are to be evaluated
all_embeddings_path = EU.all_files_in_path(embedding_storage[method], task)

In [166]:
path = all_embeddings_path[2]
path

'../ProNE/emb/kraken_bin1000_spectral.emb'

In [167]:
model = KeyedVectors.load_word2vec_format(path)
table_name = path.split("/")[-1][:-4]
if "_sparse" in table_name or "_spectral" in table_name:
    table_name = "_".join(table_name.split("_")[:-1])
model_dict_path = "../graph/{}/{}.dict".format(task, table_name)
        
print(model_dict_path)
# Obtain textified & quantized data
training_loss = []
testing_loss = []
for i in [1]:
    df_textified = EU.textify_df(
        trimmed_table, strategies, location_processed)
    x_vec = EU.vectorize_df(
        df_textified, model, model.vocab,
        model_dict=model_dict_path, model_type=method
    )

    model_2dim = EU.get_PCA_for_embedding(model, ndim=i)
    x_vec_2dim = EU.vectorize_df(
        df_textified, model_2dim, model.vocab,
        model_dict=model_dict_path, model_type=method
    )
    tests = train_test_split(x_vec_2dim, Y, test_size=test_size, random_state=1234)
    train_loss, test_loss = EU.classification_task_nn(*tests)
    training_loss.append(train_loss)
    testing_loss.append(test_loss)

../graph/kraken/kraken_bin1000.dict
Epoch 1/100
25/25 [==============================] - 0s 6ms/step - loss: 0.6879 - accuracy: 0.5530 - val_loss: 0.6812 - val_accuracy: 0.5750
Epoch 2/100
25/25 [==============================] - 0s 2ms/step - loss: 0.6882 - accuracy: 0.5553 - val_loss: 0.6814 - val_accuracy: 0.5750
Epoch 3/100
25/25 [==============================] - 0s 2ms/step - loss: 0.6795 - accuracy: 0.5875 - val_loss: 0.6820 - val_accuracy: 0.5750
Epoch 4/100
25/25 [==============================] - 0s 2ms/step - loss: 0.6879 - accuracy: 0.5527 - val_loss: 0.6814 - val_accuracy: 0.5750
Epoch 5/100
25/25 [==============================] - 0s 3ms/step - loss: 0.6885 - accuracy: 0.5529 - val_loss: 0.6814 - val_accuracy: 0.5750
Epoch 6/100
25/25 [==============================] - 0s 2ms/step - loss: 0.6865 - accuracy: 0.5611 - val_loss: 0.6815 - val_accuracy: 0.5750
Epoch 7/100
25/25 [==============================] - 0s 3ms/step - loss: 0.6787 - accuracy: 0.5882 - val_loss: 0.6818 

25/25 [==============================] - 0s 2ms/step - loss: 0.6847 - accuracy: 0.5655 - val_loss: 0.6823 - val_accuracy: 0.5750
Epoch 59/100
25/25 [==============================] - 0s 2ms/step - loss: 0.6740 - accuracy: 0.6016 - val_loss: 0.6828 - val_accuracy: 0.5750
Epoch 60/100
25/25 [==============================] - 0s 3ms/step - loss: 0.6766 - accuracy: 0.5994 - val_loss: 0.6831 - val_accuracy: 0.5750
Epoch 61/100
25/25 [==============================] - 0s 2ms/step - loss: 0.6828 - accuracy: 0.5745 - val_loss: 0.6827 - val_accuracy: 0.5750
Epoch 62/100
25/25 [==============================] - 0s 2ms/step - loss: 0.6953 - accuracy: 0.5242 - val_loss: 0.6827 - val_accuracy: 0.5750
Epoch 63/100
25/25 [==============================] - 0s 2ms/step - loss: 0.6918 - accuracy: 0.5385 - val_loss: 0.6825 - val_accuracy: 0.5750
Epoch 64/100
25/25 [==============================] - 0s 2ms/step - loss: 0.6857 - accuracy: 0.5647 - val_loss: 0.6824 - val_accuracy: 0.5750
Epoch 65/100
25/25 

In [135]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

NameError: name 'history' is not defined